In [9]:
import cv2
import numpy as np
import tensorflow as tf

In [10]:
category2label = {0: 'without', 1: 'with', 2: 'incorrect'} 
img_size=100;

In [11]:
model = tf.keras.models.load_model("mymodel.h5")

In [12]:
colors = {0: (0, 0, 255), 1: (0, 255, 0), 2: (0, 255, 255)} # les codes couleurs

# Le classifieur du détection du visage
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Vidéo du webcam
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap=cv2.VideoCapture(0)
if not cap.isOpened():
    print('ERROR')
    
    
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # convertir la frame en nuage_de_gris
    
    faces = face_cascade.detectMultiScale(gray, 1.2, 4) # détecter les visages dans la frame
    
    for (x, y, w, h) in faces:
        roi =  frame[y: y+h+100, x-30 : x+w+100]
        data = cv2.resize(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB), (img_size,img_size))
        data = data / 255.
        data = data.reshape((1,) + data.shape)
        scores = model.predict(data)
        print(scores)
        target = np.argmax(scores, axis=1)[0]
        if scores[0,target] >0.7: 
            # Dessiner les boxes
            cv2.rectangle(img=frame, pt1=(x-30, y), pt2=(x+w+100, y+h+100), color=colors[target], thickness=2)
            text = "{}: {:.2f}".format(category2label[target]+" mask", scores[0][target])
            cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    # Afficher le résultat
    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'): # entrer la touche q pour quitter
        break

cap.release()
cv2.destroyAllWindows()